In [ ]:
##label inference based on word vectors and image clustering
##hints:
##if you intend to use word embedding, please refer to the script text_processing_word_embedding to train
##if you prefer to GloVe, please refer to (https://github.com/stanfordnlp/GloVe)

In [7]:
import numpy as np
vectors=np.load("word_vector.npy")
dictionary=np.load("dictionary.npy").item()
reverse_dictionary=np.load("reverse_dictionary.npy").item()

In [8]:
## load captions
f=open("annotations/captions_on_images.txt")
text=f.readlines()
num=0;
cap={}
for line in text:
    if (num==0):
        im_id=int(line)
        cap[im_id]=""
        num=1
        continue
    if (line[0]=='*'):
        num=0
        continue
    cap[im_id]+=line
f.close()

In [12]:
##load labels
labels=[
'airplane',
 'apple',
 'backpack',
 'banana',
 'bat',
 'glove',
 'bear',
 'bed',
 'bench',
 'bicycle',
 'bird',
 'boat',
 'book',
 'bottle',
 'bowl',
 'broccoli',
 'bus',
 'cake',
 'car',
 'carrot',
 'cat',
 'phone',
 'chair',
 'clock',
 'couch',
 'cow',
 'cup',
 'table',
 'dog',
 'donut',
 'elephant',
 'hydrant',
 'fork',
 'frisbee',
 'giraffe',
 'drier',
 'handbag',
 'horse',
 'hot',
 'keyboard',
 'kite',
 'knife',
 'laptop',
 'microwave',
 'motorcycle',
 'mouse',
 'orange',
 'oven',
 'parking',
 'person',
 'pizza',
 'plant',
 'refrigerator',
 'remote',
 'sandwich',
 'scissors',
 'sheep',
 'sink',
 'skateboard',
 'skis',
 'snowboard',
 'spoon',
 'ball',
 'stop',
 'suitcase',
 'surfboard',
 'teddy',
 'tennis',
 'tie',
 'toaster',
 'toilet',
 'toothbrush',
 'traffic',
 'train',
 'truck',
 'tv',
 'umbrella',
 'vase',
 'wine',
 'zebra']

In [14]:
##load stopwords
f=open("annotations/stopwords.txt")
text=f.readlines()
stw=set()
for line in text:
    stw.add(line[:-1])

In [15]:
##labels id in dictionary
l_id=[]
for x in labels:
    l_id.append(dictionary[x])

In [16]:
##compute the label distribution on text for each original image
l_dis={}##label distribution on captions
for idx in cap.keys():
    lis=cap[idx].split()
    vote=[0]*len(labels)
    for word in lis:
        ##clean words and only consider those which are not stop words
        w="".join(l for l in word if l not in {',','.','!','?','\"','(',')','\''})
        if w in stw:
            continue
        try:
            w_id=dictionary[w]
        except:
            continue
        vec=vectors[w_id]
        ##distribution for each word
        v1=[0]*len(labels)
        for i in range(0,len(labels)):
            cate=c_id[i]
            vec2=vectors[cate]
            sim_vec=vec*vec2
            cos_sim=sum(sim_vec)
            v1[i]=cos_sim
        nsum=sum(v1)
        for i in range(0,len(labels)):
            vote[i]+=v1[i]/nsum
    nsum=sum(vote)
    l_dis[idx]=np.array(vote)/nsum

In [23]:
##compute the label distribution on clusters
num=0
c_dis={}##label distribution of clusters 
n_clu=100##number of clusters
c_dic={}
for i in range(0,n_clu):
    c_dis[i]=[0]*len(labels)
inp=open("clusters.txt")
for line in inp.readlines():
    my=line.split()
    c_dic[my[0]]=int(my[1])
inp.close()
for img in c_dic:
    sup=img.find("_")
    idx=int(img[:sup])
    cluster=c_dic[img]
    for i in range(0,len(labels)):
        c_dis[cluster][i]+=l_dis[idx][i]
for i in range(0,n_clu):
    c_dis[i]=np.array(c_dis[i])
    nsum=sum(c_dis[i])
    if nsum>0:
        c_dis[i]=c_dis[i]/nsum

In [28]:
##scoring
alpha=0.5
final_score={}
for img in c_dic:
    sup=img.find("_")
    idx=int(img[:sup])
    cluster=c_dic[img]
    final_score[img]=alpha*c_dis[cluster]+(1-alpha)*l_dis[idx]

In [31]:
##label with highest final score for one image
final_score[img].argmax()

53